# Top 250 Football transfers from 2000 to 2018

In [29]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import ColumnDataSource
from bokeh.models import HoverTool, ColorBar
from bokeh.layouts import row, column, gridplot
from bokeh.transform import factor_cmap

In [99]:
tr = pd.read_csv('top250-00-19.csv')

In [100]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4700 entries, 0 to 4699
Data columns (total 10 columns):
Name            4700 non-null object
Position        4700 non-null object
Age             4700 non-null int64
Team_from       4700 non-null object
League_from     4700 non-null object
Team_to         4700 non-null object
League_to       4700 non-null object
Season          4700 non-null object
Market_value    3440 non-null float64
Transfer_fee    4700 non-null int64
dtypes: float64(1), int64(2), object(7)
memory usage: 367.3+ KB


In [4]:
tr.head()

,Name,Position,Age,Team_from,League_from,Team_to,League_to,Season,Market_value,Transfer_fee
0,Luís Figo,Right Winger,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,2000-2001,NaN,60000000
1,Hernán Crespo,Centre-Forward,25,Parma,Serie A,Lazio,Serie A,2000-2001,NaN,56810000
2,Marc Overmars,Left Winger,27,Arsenal,Premier League,FC Barcelona,LaLiga,2000-2001,NaN,40000000
3,Gabriel Batistuta,Centre-Forward,31,Fiorentina,Serie A,AS Roma,Serie A,2000-2001,NaN,36150000
4,Nicolas Anelka,Centre-Forward,21,Real Madrid,LaLiga,Paris SG,Ligue 1,2000-2001,NaN,34500000


In [40]:
league_team = tr.groupby(by = ['League_from', 'Team_from'], as_index=False).first()[['League_from', 'Team_from']]

In [61]:
team_country = pd.read_csv('team_country.csv', delimiter=';')
team_country_dict = team_country[['Country','Team']].to_dict()

In [101]:
tr= pd.merge(left=tr, right=team_country, on=None, left_on='Team_from', right_on='Team').drop('Team', axis = 1)

In [109]:
tr= pd.merge(left=tr, right=team_country, on=None, left_on='Team_to', right_on='Team').drop('Team', axis = 1)

In [120]:
tr_col = ['Name', 'Position', 'Age', 'Team_from', 'League_from', 'Team_to',
       'League_to', 'Season', 'Market_value', 'Transfer_fee', 'Country_from',
       'Country_to']

In [121]:
tr.columns = tr_col

In [123]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27511 entries, 0 to 27510
Data columns (total 12 columns):
Name            27511 non-null object
Position        27511 non-null object
Age             27511 non-null int64
Team_from       27511 non-null object
League_from     27511 non-null object
Team_to         27511 non-null object
League_to       27511 non-null object
Season          27511 non-null object
Market_value    19377 non-null float64
Transfer_fee    27511 non-null int64
Country_from    27511 non-null object
Country_to      27511 non-null object
dtypes: float64(1), int64(2), object(9)
memory usage: 2.7+ MB


In [43]:
league_team.to_excel('league_team.xlsx')

In [129]:
tr['transfer_fee_M']= tr['Transfer_fee'].map(lambda x : str(x/1000000)+' M')

In [130]:
tr['transfer_fee_M'].head()

0    60.0 M
1    25.0 M
2    25.0 M
3    25.0 M
4    25.0 M
Name: transfer_fee_M, dtype: object

In [131]:
tr['Year']= tr['Season'].map(lambda x : x.split('-')[0])
tr['Year'] = tr['Year'].astype(int)

In [132]:
tr = tr.sort_values('Transfer_fee', ascending=False)

In [133]:
tr_yearly_10 = tr.groupby('Year').head(10).reset_index(drop=True)
tr_max_100 = tr.head(100).reset_index(drop=True)
tr_max_20 = tr.head(20).reset_index(drop=True)

In [139]:
source1 = ColumnDataSource(tr_max_20)
p1 = figure(plot_width=800, plot_height=500,x_axis_label='Year', 
            y_axis_label='Transfer Fee', tools='box_select, lasso_select',)
p1.circle('Year', 'Transfer_fee', size=8, source=source1, 
            selection_color='red',
            nonselection_fill_alpha=0.2, nonselection_fill_color='grey',
            hover_fill_color='firebrick', hover_alpha=1,
            hover_line_color='red')
hover = HoverTool(tooltips = [('Player', '@Name'), 
                              ('From (Team)', ('@Team_from')),('To (Team)', '@Team_to'), 
                              ('From (Country)', '@Country_from'), ('To (Country)', '@Country_to'),
                              ('Transfer Fee', '@transfer_fee_M'), ('Year', '@Year')])

p1.add_tools(hover)

In [135]:
source2 = ColumnDataSource(tr_yearly_10)
p2 = figure(plot_width=800, plot_height=500,
            x_axis_label='Year', y_axis_label='Transfer Fee', tools='box_select, lasso_select',)
p2.circle('Year', 'Transfer_fee', size=8, source=source2, 
            selection_color='red',
            nonselection_fill_alpha=0.2, nonselection_fill_color='grey',
            hover_fill_color='firebrick', hover_alpha=1,
            hover_line_color='red')
hover = HoverTool(tooltips = [('Player', '@Name'), 
                              ('From (Team)', '@Team_from'),('To (Team)', '@Team_to'), 
                              ('From (Country)', '@Country_from'), ('To (Countru)', '@Country_to'),
                              ('Transfer Fee', '@transfer_fee_M'), ('Year', '@Year')])
p2.add_tools(hover)

In [136]:
from bokeh.palettes import Category20_12
from bokeh.models import FactorRange, ColorBar
from bokeh.transform import linear_cmap
from math import pi

group = tr_max_100.head(10).sort_values('Year')
source3= ColumnDataSource(group)
player_list = group.Name.unique().tolist()
p3 = figure(plot_width=800, plot_height=500, title="Most Expensive 100 Transfers",
           x_range=FactorRange(factors=player_list), toolbar_location=None, tools="")

p3.xgrid.grid_line_color = None
p3.xaxis.axis_label = "Most Expensive Players"
p3.xaxis.major_label_orientation = 1.2
p3.xaxis.major_label_text_color = "SeaGreen"
p3.xaxis.major_label_orientation = pi/4
p3.xaxis.major_label_text_font_size = '12pt'



index_cmap = factor_cmap('Position', palette=Category20_12, 
                         factors=player_list)

#Use the field name of the column source
mapper = linear_cmap(field_name='Transfer_fee', palette=Category20_12 ,
                     low=min(tr_max_100['Transfer_fee']),high=max(tr_max_100['Transfer_fee']))

p3.circle(x='Name', y='Transfer_fee', source=source3,
       line_color=mapper, fill_color=mapper, size = 20,
       hover_line_color="darkgrey", hover_fill_color=index_cmap)



hover = HoverTool(tooltips = [('Player', '@Name'), 
                              ('From (Team)', '@Team_from'),('To (Team)', '@Team_to'), 
                              ('From (Country)', '@Country_from'), ('To (Countru)', '@Country_to'),
                              ('Transfer Fee', '@transfer_fee_M'), ('Year', '@Year')])

#color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
#p3.add_layout(color_bar, 'right')

p3.add_tools(hover)

#outline_border
p3.outline_line_width = 7
p3.outline_line_alpha = 0.3
p3.outline_line_color = "navy"

#show(p3)

In [137]:
# Import Panel from bokeh.models.widgets
from bokeh.models.widgets import Panel

# Create tab1 from plot p1: tab1
tab1 = Panel(child=p1, title='Maximum 20 Transfer Fee')

# Create tab2 from plot p2: tab2
tab2 = Panel(child=p2, title='Max 10 Transfer Fee (Yearly)')

# Create tab2 from plot p2: tab2
tab3 = Panel(child=p3, title='Max 10 Transfer Fee')

# Import Tabs from bokeh.models.widgets
from bokeh.models.widgets import Tabs

# Create a Tabs layout: layout
layout = Tabs(tabs=[tab1, tab2, tab3])

# Specify the name of the output_file and show the result
output_notebook()
show(layout)

Loading BokehJS ...